In [1]:
import sys
sys.path.insert(0, "../../")
from src.data import make_dataset
from src.models import rnn_models, metrics
from pathlib import Path
from datetime import datetime
import gin
import ray

In [2]:
data_dir = Path("../../data/external/gestures-dataset/").resolve()
trainloader, testloader = make_dataset.get_gestures(data_dir=data_dir, split=0.8, batchsize=32)

100%|██████████| 651/651 [00:00<00:00, 2144.33it/s]


In [3]:
x, y = next(iter(trainloader))
x.shape, y.shape

(torch.Size([32, 32, 3]), torch.Size([32]))

In [4]:
accuracy = metrics.Accuracy()

In [5]:
config = {
    "input_size" : 3,
    "hidden_size" : 32, 
    "dropout" : 0.1,
    "num_layers": 2,
    "output_size" : 20,
}

In [6]:
gin.parse_config_file("gestures.gin")

timestamp = datetime.now().strftime("%Y%m%d-%H%M")
log_dir = Path("../../models/gestures/" + timestamp).resolve()
if not log_dir.exists():
    log_dir.mkdir(parents=True)


model = rnn_models.GRUmodel(config)

model = rnn_models.trainloop(
    model=model,
    metrics=[accuracy],
    train_dataloader=trainloader,
    test_dataloader=testloader,
    log_dir=log_dir,
)

2022-05-16 21:21:31.286 | INFO     | src.data.data_tools:clean_dir:167 - Clean out /Users/rgrouls/code/deep_learning/models/gestures/20220516-2121
100%|██████████| 50/50 [00:48<00:00,  1.03it/s]


In [9]:
from ray.tune import JupyterNotebookReporter


tune_dir = Path("../../models/ray/").resolve()
tune_dir.exists(), tune_dir, data_dir.exists(), data_dir

(True,
 PosixPath('/Users/rgrouls/code/deep_learning/models/ray'),
 True,
 PosixPath('/Users/rgrouls/code/deep_learning/data/external/gestures-dataset'))

In [10]:
from ray import tune

In [11]:
def train(config):
    trainloader, testloader = make_dataset.get_gestures(data_dir=config["data_dir"], split=0.8, batchsize=32)
    model = rnn_models.GRUmodel(config)

    model = rnn_models.trainloop(
        model=model,
        metrics=[accuracy],
        train_dataloader=trainloader,
        test_dataloader=testloader,
        log_dir=config["tune_dir"],
    )

In [9]:
import ray
import torch
ray.init()

RayContext(dashboard_url='', python_version='3.8.13', ray_version='1.12.0', ray_commit='f18fc31c7562990955556899090f8e8656b48d2d', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-05-16_16-47-38_668407_4329/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-05-16_16-47-38_668407_4329/sockets/raylet', 'webui_url': '', 'session_dir': '/tmp/ray/session_2022-05-16_16-47-38_668407_4329', 'metrics_export_port': 58677, 'gcs_address': '127.0.0.1:50588', 'address': '127.0.0.1:50588', 'node_id': '256239ae12c4caefc4212ec704b718c308336b28465ea6b1e2ce8c96'})

In [ ]:
reporter = JupyterNotebookReporter(overwrite=True)

config = {
    "input_size" : 3,
    "hidden_size" : tune.randint(16, 128), 
    "dropout" : 0.1,
    "num_layers": 2,
    "output_size" : 20,
    "tune_dir" : tune_dir,
    "data_dir" : data_dir,
}

analysis = tune.run(train,
         config = config,
         metric="test_loss",
         mode="min",
         local_dir=tune_dir,
         progress_reporter=reporter,
         num_samples=4,
         stop={"training_iteration": 50},
         verbose=1)

In [19]:
ray.shutdown()